# 【問題1】コンペティション内容の確認

## コンペティションのOverviewページ読み、「Home Credit Default Risk」について以下の観点について確認してください。

# 何を学習し、何を予測するのか

 - クライアントの特徴項目を学習し、返済能力を予測する。

## どのようなファイルを作りKaggleに提出するか

- 特徴量同士の関係性を調べ、返済能力に関わりの深い項目を抜擢し、最終的な見解を加えて提出

## 提出されたものはどういった指標値で評価されるのか

- 提出物は 、予測された確率と観察されたターゲットの間のROC曲線の下の領域で評価されます。

まずは単純な方法による ベースラインモデル　を作成します。精度の基準となるモデルです。
精度を高くする必要はありません。エラーなく実行でき、Kaggle側に推定値を提出できることを目指します。

# 【問題2】学習と検証

データを簡単に分析、前処理し、学習、検証するまでの一連の流れを作成・実行してください。
検証にはこのコンペティションで使用される評価指標を用いるようにしてください。学習に用いる手法は指定しません。

データの分析
　　目的変数に関わりの深い項目をピックアップし、検証する
  　二値分類か多値分類か選択する
前処理
　　ピックアップした項目を加工する
    訓練用にデータを分割する
学習方法を選び、必要なら標準化をする
　　検証データをメソッドを使い推定する
　　検証データの特徴量を入力して得られた推定値と、検証データの正解値の近さを計算し、学習したモデルを評価します
必要に応じて可視化する
ROC曲線を作成
　　予測された確率と観察されたターゲットの間のROC曲線の下の領域で評価される
テストデータに対して推定を行い、Kaggleに提出を行う

In [1]:
#主要なものをインポートする
import missingno as msno
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import warnings #ワーニング関連のモジュール？
warnings.filterwarnings('ignore') #ワーニングが消える？


df = pd.read_csv("application_train.csv")
#検証
df
#ROC曲線を描くには、真陽性率（TPR）と偽陽性率（FPR）のみが必要です
#TPRは、テスト中に利用可能なすべての陽性サンプルの中で発生する正しい陽性結果の数を定義します
#FPRは、テスト中に利用可能なすべての陰性サンプルの中で発生する誤った陽性結果の数を

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,456251,0,Cash loans,M,N,N,0,157500.0,254700.0,27558.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
307507,456252,0,Cash loans,F,N,Y,0,72000.0,269550.0,12001.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
307508,456253,0,Cash loans,F,N,Y,0,153000.0,677664.0,29979.0,...,0,0,0,0,1.0,0.0,0.0,1.0,0.0,1.0
307509,456254,1,Cash loans,F,N,Y,0,171000.0,370107.0,20205.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


## データフレームの情報

In [2]:
#情報
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307511 entries, 0 to 307510
Columns: 122 entries, SK_ID_CURR to AMT_REQ_CREDIT_BUREAU_YEAR
dtypes: float64(65), int64(41), object(16)
memory usage: 286.2+ MB


In [3]:
#大きさ
df.shape

(307511, 122)

In [4]:
#確認
df.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


## 前処理

AMT_INCOME_TOTAL,AMT_CREDIT,AMT_GOODS_PRICEを比較特徴量として抜き出す。

In [5]:
#抜き出し
df_selected = df.loc[:, ["AMT_INCOME_TOTAL", "AMT_CREDIT","TARGET"]]#, "AMT_GOODS_PRICE"
df_selected.head()

,AMT_INCOME_TOTAL,AMT_CREDIT,TARGET
0,202500.0,406597.5,1
1,270000.0,1293502.5,0
2,67500.0,135000.0,0
3,135000.0,312682.5,0
4,121500.0,513000.0,0


欠損値のある行を削除

In [6]:
#削除
df_selected = df_selected.dropna(how='any')
df_selected.head()
df_selected.shape

(307511, 3)

確認

In [7]:
#確認
missing_data = pd.DataFrame(df_selected.isnull().sum())
missing_data

,0
AMT_INCOME_TOTAL,0
AMT_CREDIT,0
TARGET,0


## 学習モデル

In [8]:
# sklearnのから各種モデルをimportする
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# データの前処理
# データを分割
X_train, X_test, y_train, y_test = train_test_split(df_selected.iloc[:, :-1], df_selected.loc[:, "TARGET"], test_size=0.25, random_state=42)

# データを確認
print("X_train.shape : {}, X_test.shape : {}".format(X_train.shape, X_test.shape))
print("y_train.shape : {}, y_test.shape : {}".format(y_train.shape, y_test.shape))


X_train.shape : (230633, 2), X_test.shape : (76878, 2)
y_train.shape : (230633,), y_test.shape : (76878,)


## 最近傍法

In [9]:
# sklearnから標準化をするモデルをインポートする
from sklearn.preprocessing import StandardScaler
# モデルをインスタンス化して、モデルの中にあるメソッドを使える状態にする。
scaler = StandardScaler()

# 標準化をする（訓練用データで.fitを行う）
scaler.fit(X_train)

# 標準化をする（訓練用、検証用双方で.transformを行う）
X_train_transformed = scaler.transform(X_train)
X_test_transformed = scaler.transform(X_test)

In [10]:
# sklearnから最近傍法をするモデルをインポートする
from sklearn.neighbors import KNeighborsClassifier

# モデルに引数を渡してインスタン化する
neigh_1 = KNeighborsClassifier(n_neighbors=1)
neigh_3 = KNeighborsClassifier(n_neighbors=3)
neigh_5 = KNeighborsClassifier(n_neighbors=5)

# 学習する（訓練用データを.fitする）
neigh_1.fit(X_train_transformed, y_train)
neigh_3.fit(X_train_transformed, y_train)
neigh_5.fit(X_train_transformed, y_train)

# 推定する（検証用データを.predictする）
y_pred_neigh_1 = neigh_1.predict(X_test_transformed)
y_pred_neigh_3 = neigh_3.predict(X_test_transformed)
y_pred_neigh_5 = neigh_5.predict(X_test_transformed)

In [11]:
# モデルmetrics をimportすればaccuracy等を一つずつインポートしなくても良い
from sklearn import metrics 

# 評価する関数を作成する
def evaluate(y_true, y_pred):
    """
    2値分類の評価指標を計算する
    """
    acc = metrics.accuracy_score(y_test, y_pred)
    precision = metrics.precision_score(y_test, y_pred)
    recall = metrics.recall_score(y_test, y_pred)
    f1 = metrics.f1_score(y_test, y_pred)
    confusion = metrics.confusion_matrix(y_test, y_pred)
    return acc, precision, recall, f1, confusion

print("1-nn")
print("acc : {}\nprecision : {}\nrecall : {}\nf1 : {}\nconfusion matrix \n {}".format(*evaluate(y_test, y_pred_neigh_1)))
print("------")

print("3-nn")
print("acc : {}\nprecision : {}\nrecall : {}\nf1 : {}\nconfusion matrix \n {}".format(*evaluate(y_test, y_pred_neigh_3)))
print("------")

print("5-nn")
print("acc : {}\nprecision : {}\nrecall : {}\nf1 : {}\nconfusion matrix \n {}".format(*evaluate(y_test, y_pred_neigh_5)))

1-nn
acc : 0.8584640599391243
precision : 0.09555018972059331
recall : 0.08948473590696172
f1 : 0.09241804987905582
confusion matrix 
 [[65443  5244]
 [ 5637   554]]
------
3-nn
acc : 0.9009599625380473
precision : 0.1172673480365788
recall : 0.03521240510418349
f1 : 0.054161490683229806
confusion matrix 
 [[69046  1641]
 [ 5973   218]]
------
5-nn
acc : 0.9152553396290226
precision : 0.10294117647058823
recall : 0.006784041350347278
f1 : 0.01272920139415063
confusion matrix 
 [[70321   366]
 [ 6149    42]]
